# Style Instruction Evaluation Framework

This notebook evaluates how well derived style instructions enable style replication, compared to alternative approaches.

## Methodology

For each gold standard text:
1. **Flatten**: Extract content while removing style
2. **Reconstruct**: Generate text using 4 different methods (M stochastic runs each):
   - Generic baseline
   - Few-shot learning
   - Author name prompting
   - Derived style instructions
3. **Judge (Blind Comparative)**: Judge ranks all 4 reconstructions from 1-4 based on similarity to original
   - **Blind evaluation**: Judge sees only anonymous labels (Text A, B, C, D) - no method names
   - **Ranking**: 1 = most similar, 2 = second, 3 = third, 4 = least similar
   - **Order randomized**: Position of methods varies across samples to eliminate bias
4. **Aggregate**: Analyze rankings to determine which method best captures style

## Key Features

- **Crash resilient**: All LLM responses saved to SQLite immediately
- **Resume support**: Can restart after failures, skips completed work
- **Blind evaluation**: Eliminates judge bias by hiding method names
- **Comparative ranking**: More informative than binary comparisons

### Install Libraries and Check

In [81]:
!pip install -r requirements.txt


[notice] A new release of pip is available: 24.1.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [82]:
try:
    import litellm
    print('Providers\n=========')
    print('* ' + '\n* '.join(litellm.LITELLM_CHAT_PROVIDERS))
    litellm.drop_params = True
except ImportError as e:
    print(f"✗ Cannot import litellm: {e}")

Providers
* openai
* openai_like
* bytez
* xai
* custom_openai
* text-completion-openai
* cohere
* cohere_chat
* clarifai
* anthropic
* anthropic_text
* replicate
* huggingface
* together_ai
* datarobot
* openrouter
* cometapi
* vertex_ai
* vertex_ai_beta
* gemini
* ai21
* baseten
* azure
* azure_text
* azure_ai
* sagemaker
* sagemaker_chat
* bedrock
* vllm
* nlp_cloud
* petals
* oobabooga
* ollama
* ollama_chat
* deepinfra
* perplexity
* mistral
* groq
* nvidia_nim
* cerebras
* baseten
* ai21_chat
* volcengine
* codestral
* text-completion-codestral
* deepseek
* sambanova
* maritalk
* cloudflare
* fireworks_ai
* friendliai
* watsonx
* watsonx_text
* triton
* predibase
* databricks
* empower
* github
* custom
* litellm_proxy
* hosted_vllm
* llamafile
* lm_studio
* galadriel
* gradient_ai
* github_copilot
* novita
* meta_llama
* featherless_ai
* nscale
* nebius
* dashscope
* moonshot
* v0
* heroku
* oci
* morph
* lambda_ai
* vercel_ai_gateway
* wandb
* ovhcloud
* lemonade


## Setup and Configuration

In [83]:
import os
import json
from pathlib import Path
import pandas as pd
from datetime import datetime

### Initialize Base Objects
The base objects part of the current project library (`belletrist`) are initialized. They are:
* `LLM`: the LLM object.
* `LLMConfig`: the configuration of the LLM object, such as what model to use.
* `PromptMaker`: generates prompts from templates and variables
* `DataSampler`: retrieves and samples text at a source directory

These will implement text transformations by LLMs part of the evaluation process. They build on the third-party LLMs, which we furthermore split into LLMs for text reconstruction and text judging, the key parameters for which are set below.

In [84]:
model_reconstruction_string = 'together_ai/Qwen/Qwen3-235B-A22B-Instruct-2507-tput'
model_reconstruction_api_key_env_var = 'TOGETHER_AI_API_KEY'
#model_reconstruction_string = 'anthropic/claude-sonnet-4-5-20250929'
#model_reconstruction_api_key_env_var = 'ANTHROPIC_API_KEY'
#model_reconstruction_string = 'openai/gpt-5.1-2025-11-13'
#model_reconstruction_api_key_env_var = 'OPENAI_API_KEY'
#model_reconstruction_string = 'together_ai/moonshotai/Kimi-K2-Instruct'
#model_reconstruction_api_key_env_var = 'TOGETHER_AI_API_KEY'
#model_reconstruction_string = 'mistral/mistral-large-2512'
#model_reconstruction_api_key_env_var = 'MISTRAL_API_KEY'
#model_judge_string = 'anthropic/claude-sonnet-4-5-20250929'
#model_judge_api_key_env_var = 'ANTHROPIC_API_KEY'
model_judge_string = 'mistral/mistral-large-2512'
model_judge_api_key_env_var = 'MISTRAL_API_KEY'

In [ ]:
from belletrist import PromptMaker, DataSampler, StyleEvaluationStore, SegmentStore

prompt_maker = PromptMaker()
sampler = DataSampler(
    data_path=(Path(os.getcwd()) / "data" / "russell").resolve()
)

# Configure methods for this experiment: agent_fewshot instead of instructions
METHODS = ['generic', 'fewshot', 'author', 'agent_fewshot']

# Segment store database path (for agent_fewshot retrieval)
DB_PATH = Path(__file__).parent / "segments.db" if '__file__' in globals() else Path("segments.db")

# Initialize store with configured methods
store = StyleEvaluationStore(
    Path("style_eval_agent_fewshot.db"),
    methods=METHODS
)

print(f"✓ Configured methods: {METHODS}")
print(f"✓ Store initialized at {store.filepath}")
print(f"✓ Segment database: {DB_PATH}")

In [86]:
store.reset('judgments_only')

In [87]:
from belletrist import LLM, LLMConfig

reconstruction_llm = LLM(LLMConfig(
    model=model_reconstruction_string,
    api_key=os.environ.get(model_reconstruction_api_key_env_var)
))
judge_llm = LLM(LLMConfig(
    model=model_judge_string,
    api_key=os.environ.get(model_judge_api_key_env_var)
))

### Sample Test Data and Few-Shot Data
The reconstruction method tests build on gold standard texts. The test also includes few-shot prompting with the gold standard texts. In order to not skew the tests, no few-shot examples can overlap with the test texts.

In [88]:
n_sample = 5
m_paragraphs_per_sample = 5
n_few_shot_sample = 5

In [89]:
#test_texts = []
#for _ in range(n_sample):
#    test_texts.append(sampler.sample_segment(p_length=m_paragraphs_per_sample))

In [90]:
quality_texts_deterministic = [
    sampler.get_paragraph_chunk(file_index=0, paragraph_range=slice(9, 9+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=0, paragraph_range=slice(29, 29+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=0, paragraph_range=slice(131, 131+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=1, paragraph_range=slice(13, 13+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=1, paragraph_range=slice(39, 39+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=1, paragraph_range=slice(192, 192+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=2, paragraph_range=slice(20, 20+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=2, paragraph_range=slice(43, 43+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=2, paragraph_range=slice(146, 146+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=3, paragraph_range=slice(7, 7+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=3, paragraph_range=slice(73, 73+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=3, paragraph_range=slice(202, 202+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=3, paragraph_range=slice(285, 285+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=4, paragraph_range=slice(4, 4+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=4, paragraph_range=slice(67, 67+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=4, paragraph_range=slice(124, 124+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=5, paragraph_range=slice(6, 6+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=5, paragraph_range=slice(119, 119+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=5, paragraph_range=slice(301, 301+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=6, paragraph_range=slice(23, 23+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=6, paragraph_range=slice(75, 75+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=6, paragraph_range=slice(152, 152+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=6, paragraph_range=slice(198, 198+m_paragraphs_per_sample)),
    sampler.get_paragraph_chunk(file_index=6, paragraph_range=slice(271, 271+m_paragraphs_per_sample)),
]
reindex = [0, 5, 10, 15, 20, 1, 6, 11, 16, 21, 2, 7, 12, 17, 22, 3, 8, 13, 18, 23, 4, 9, 14, 19]
test_texts = [
    quality_texts_deterministic[i] for i in reindex[:n_sample]
]

In [91]:
#few_shot_texts = []
#while len(few_shot_texts) < n_few_shot_sample:
#    p = sampler.sample_segment(p_length=m_paragraphs_per_sample)

    # Check if p overlaps with any test text
    # Two segments overlap if they're from the same file AND their paragraph ranges overlap
    # Ranges [a, b) and [c, d) overlap if: a < d AND c < b
#    has_overlap = any(
#        p.file_index == test_seg.file_index and
#        p.paragraph_start < test_seg.paragraph_end and
#        test_seg.paragraph_start < p.paragraph_end
#        for test_seg in test_texts
#    )
#
#    if not has_overlap:
#        few_shot_texts.append(p)

In [92]:
#few_shot_texts = [
#    quality_texts_deterministic[i] for i in reversed(reindex[:n_few_shot_sample])
#]

In [93]:
sampler_fake = DataSampler(
    data_path=(Path(os.getcwd()) / "data" / "other_author").resolve()
)
few_shot_texts = [
    sampler_fake.get_paragraph_chunk(file_index=0, paragraph_range=slice(0, 5)),
    sampler_fake.get_paragraph_chunk(file_index=0, paragraph_range=slice(10, 15)),
    sampler_fake.get_paragraph_chunk(file_index=0, paragraph_range=slice(20, 25)),
    sampler_fake.get_paragraph_chunk(file_index=0, paragraph_range=slice(30, 35)),
    sampler_fake.get_paragraph_chunk(file_index=0, paragraph_range=slice(40, 45)),
]

## Create the Test Transformation Objects
The combination of prompt and LLM leads to the following operators in the test chain:
* **Style Flattener**, which given a text compresses it into its content bare bones.
* **Reconstructor, LLM House Style**, which given a compressed content expands it into a complete text with the "house style" of the LLM employed for the reconstruction.
* **Reconstructor, Few Shot**, which given a compressed content expands it into a complete text with a few text excerpts on unrelated topics as style guide.
* **Reconstructor, LLM Author Model**, which given a compressed content expands it into a complete text with the named author's style as the LLM conceives it without any other guidance.
* **Reconstructor, Style Instruction**, which given a compressed content expands it into a complete text following the detailed style instruction, as derived from previous analysis.

In [94]:
style_instructions_path = Path("outputs/author_modeling/author_model_definition_001_sonnet.txt")

if style_instructions_path.exists():
    style_instructions = style_instructions_path.read_text()
    print(f"✓ Loaded style instructions ({len(style_instructions)} chars)")
    print(f"\nFirst 200 chars:\n{style_instructions[:500]}...")
else:
    print("⚠ Style instructions file not found. Please provide path.")
    style_instructions = ""  # Placeholder

✓ Loaded style instructions (20519 chars)

First 200 chars:
---
synthesis_id: author_model_definition_001
synthesis_type: author_model_definition
model: claude-sonnet-4-5-20250929
created_at: 2025-12-02T12:29:47.382030
parent_synthesis_id: implied_author_synthesis_001
num_samples: 5
sample_ids:
  - sample_005
  - sample_003
  - sample_002
  - sample_004
  - sample_001
is_homogeneous_model: true
models_used:
  - claude-sonnet-4-5-20250929
---

## PART 1: THE GENERATIVE STANCE

When you write as this author, you inhabit a position of **clarity achieved and...


In [ ]:
# Import from style-retrieval (no models/ subdirectory)
from belletrist.style_evaluation_models import (
    MethodMapping,
    StyleJudgmentComparative
)
from belletrist.prompts import (
    StyleFlatteningConfig,
    StyleReconstructionGenericConfig,
    StyleReconstructionFewShotConfig,
    StyleReconstructionAuthorConfig,
    StyleJudgeComparativeConfig
)

# Configuration
n_runs = 2
n_judge_runs = 1
AUTHOR_NAME = "Bertrand Russell"

# Reconstructor configs (agent_fewshot handled separately)
RECONSTRUCTORS_CFGS = {
    'generic': StyleReconstructionGenericConfig,
    'fewshot': StyleReconstructionFewShotConfig,
    'author': StyleReconstructionAuthorConfig,
    'agent_fewshot': None  # Special case: uses agent_rewriter
}

# Reconstructor kwargs
RECONSTRUCTORS_KWARGS = {
    'generic': {},
    'fewshot': {'few_shot_examples': [seg.text for seg in few_shot_texts]},
    'author': {'author_name': AUTHOR_NAME},
    'agent_fewshot': {}  # Handled separately
}

print(f"✓ Configuration: {n_runs} runs per sample, {len(METHODS)} reconstruction methods")
print(f"✓ Methods: {', '.join(METHODS)}")

## Evaluation Pipeline

### Step 1: Content Flattening

Extract content from each test sample, removing stylistic elements:

In [96]:
# Step 1: Save samples and flatten content
print("=== Step 1: Flattening and Saving Samples ===\n")

for k_text, test_sample in enumerate(test_texts):
    sample_id = f"sample_{k_text:03d}"
    
    # Skip if already saved
    if store.get_sample(sample_id):
        print(f"✓ {sample_id} already flattened (skipping)")
        continue
    
    print(f"Flattening {sample_id}...", end=" ")
    
    # Flatten content
    flatten_prompt = prompt_maker.render(
        StyleFlatteningConfig(text=test_sample.text)
    )
    flattened = reconstruction_llm.complete(flatten_prompt)
    
    # Save to store with provenance
    source_info = f"File {test_sample.file_index}, para {test_sample.paragraph_start}-{test_sample.paragraph_end}"
    store.save_sample(
        sample_id=sample_id,
        original_text=test_sample.text,
        flattened_content=flattened.content,
        flattening_model=flattened.model,
        source_info=source_info
    )
    
    print(f"✓ ({len(flattened.content)} chars)")

print(f"\n✓ All samples flattened and saved to store")

=== Step 1: Flattening and Saving Samples ===

✓ sample_000 already flattened (skipping)
✓ sample_001 already flattened (skipping)
✓ sample_002 already flattened (skipping)
✓ sample_003 already flattened (skipping)
✓ sample_004 already flattened (skipping)

✓ All samples flattened and saved to store


In [107]:
print('====ORIGINAL===')
print(store.get_sample('sample_000')['original_text'])
print('\n\n====FLATTENED====')
print(store.get_sample('sample_000')['flattened_content'])

====ORIGINAL===
In reading even the best treatises on education written in former
times, one becomes aware of certain changes that have come over
educational theory. The two great reformers of educational theory
before the nineteenth century were Locke and Rousseau. Both deserved
their reputation, for both repudiated many errors which were
wide-spread when they wrote. But neither went as far in his own
direction as almost all modern educationists go. Both, for example,
belong to the tendency which led to liberalism and democracy; yet
both consider only the education of an aristocratic boy, to which one
man’s whole time is devoted. However excellent might be the results
of such a system, no man with a modern outlook would give it serious
consideration, because it is arithmetically impossible for every child
to absorb the whole time of an adult tutor. The system is therefore
one which can only be employed by a privileged caste; in a just world,
its existence would be impossible. The mode

### Step 2: Reconstruction

Generate reconstructions using all 4 methods, with M stochastic runs each:

In [ ]:
# Step 2: Generate reconstructions (with crash resume and agent_fewshot support)
print("=== Step 2: Generating Reconstructions ===\n")

for sample_id in store.list_samples():
    sample = store.get_sample(sample_id)
    print(f"\n{sample_id}:")
    
    for run in range(n_runs):
        print(f"  Run {run}:")
        
        # Check which methods need reconstruction
        for method in METHODS:
            if store.has_reconstruction(sample_id, run, method):
                print(f"    ✓ {method:15s} (already done)")
                continue
            
            # Special handling for agent_fewshot
            if method == 'agent_fewshot':
                from belletrist.agent_rewriter import agent_rewrite
                
                # Initialize LLMs with proper temperatures for agent workflow
                planning_llm = LLM(LLMConfig(
                    model=model_reconstruction_string,
                    api_key=os.environ.get(model_reconstruction_api_key_env_var),
                    temperature=0.5,  # Deterministic planning
                    max_tokens=4096
                ))
                rewriting_llm = LLM(LLMConfig(
                    model=model_reconstruction_string,
                    api_key=os.environ.get(model_reconstruction_api_key_env_var),
                    temperature=0.7,  # Creative rewriting
                    max_tokens=8192
                ))
                
                with SegmentStore(DB_PATH) as segment_store:
                    reconstructed_text = agent_rewrite(
                        flattened_content=sample['flattened_content'],
                        segment_store=segment_store,
                        planning_llm=planning_llm,
                        rewriting_llm=rewriting_llm,
                        prompt_maker=prompt_maker
                    )
                
                # Save reconstruction
                store.save_reconstruction(
                    sample_id=sample_id,
                    run=run,
                    method=method,
                    reconstructed_text=reconstructed_text,
                    model=model_reconstruction_string
                )
                print(f"    ✓ {method:15s} ({len(reconstructed_text)} chars)")
            else:
                # Standard reconstruction
                config = RECONSTRUCTORS_CFGS[method](
                    content_summary=sample['flattened_content'],
                    **RECONSTRUCTORS_KWARGS[method]
                )
                prompt = prompt_maker.render(config)
                response = reconstruction_llm.complete(prompt)
                
                # Save immediately (crash resilient!)
                store.save_reconstruction(
                    sample_id=sample_id,
                    run=run,
                    method=method,
                    reconstructed_text=response.content,
                    model=response.model
                )
                print(f"    ✓ {method:15s} ({len(response.content)} chars)")

stats = store.get_stats()
print(f"\n✓ Generated {stats['n_reconstructions']} total reconstructions")
print(f"✓ Configured methods: {stats['configured_methods']}")

In [99]:
config = RECONSTRUCTORS_CFGS['fewshot'](
                content_summary=sample['flattened_content'],
                **RECONSTRUCTORS_KWARGS['fewshot']
            )
prompt = prompt_maker.render(config)
print(prompt)

You are given a content summary that captures the core ideas and arguments of a text, but stripped of stylistic elements.

Your task is to expand this summary into a complete prose passage using a writing style similar to the examples below.

**CRITICAL REQUIREMENTS:**
- Match the STYLE of the examples (rhythm, vocabulary, tone, sentence structure), not their formatting choices.
- If the examples are plain prose with no titles/headers/formatting, write plain prose.
- Do NOT add meta-commentary, preambles like "Here is the passage...", or postscripts explaining your work.
- Simply write naturally in this style.

**EXAMPLE TEXTS (demonstrating the target style):**

---

When the business man rebukes the idealism of his office-boy, it
is commonly in some such speech as this:  "Ah, yes, when one is young,
one has these ideals in the abstract and these castles in the air;
but in middle age they all break up like clouds, and one comes down
to a belief in practical politics, to using the mach

In [106]:
reconstructions = store.get_reconstructions('sample_000', 0)
for reconstructor in reconstructions.keys():
    print(f"{reconstructor.upper()}\n===================")
    print(f"\n{reconstructions.get(reconstructor)}\n\n")

AUTHOR

The evolution of educational theory, like that of most human institutions, has been a gradual and halting progress, marked now and then by flashes of insight from solitary thinkers who, though often constrained by the prejudices of their age, have succeeded in loosening the chains of tradition. Before the nineteenth century, it was John Locke and Jean-Jacques Rousseau who stood pre-eminent as reformers of pedagogical thought, each in his own way challenging the superstitions and cruelties that had long dominated the upbringing of the young. Locke, with his sober empiricism, insisted that the mind at birth is a *tabula rasa*, to be inscribed not by inherited doctrine but by experience and discipline; Rousseau, with a more romantic fervour, proclaimed that nature is inherently good and that the child must be allowed to develop freely, uncorrupted by the artificialities of civilization. Both men, in varying degrees, rejected the prevailing errors of their time—the blind emphasis o

### Step 3: Judging

Compare each reconstruction against the original using the judge LLM:

In [ ]:
# Import from style-retrieval (no models/ subdirectory)
from belletrist.style_evaluation_models import StyleJudgmentComparative
from belletrist.prompts import StyleJudgeComparativeConfig

# Step 3: Comparative blind judging (with crash resume and judge consistency testing)
print("=== Step 3: Comparative Blind Judging ===\n")

for sample_id in store.list_samples():
    sample = store.get_sample(sample_id)
    print(f"\n{sample_id}:")
    
    for reconstruction_run in range(n_runs):
        print(f"  Reconstruction run {reconstruction_run}:")
        
        # Get all 4 reconstructions ONCE
        reconstructions = store.get_reconstructions(sample_id, reconstruction_run)
        if len(reconstructions) != 4:
            print(f"    ✗ Missing reconstructions (found {len(reconstructions)}/4)")
            continue
        
        # Create mapping ONCE per reconstruction_run (deterministic seed for reproducibility)
        mapping = store.create_random_mapping(seed=hash(f"{sample_id}_{reconstruction_run}"))
        
        # Build prompt ONCE per reconstruction_run (SAME prompt for all judge runs)
        judge_config = StyleJudgeComparativeConfig(
            original_text=sample['original_text'],
            reconstruction_text_a=reconstructions[mapping.text_a],
            reconstruction_text_b=reconstructions[mapping.text_b],
            reconstruction_text_c=reconstructions[mapping.text_c],
            reconstruction_text_d=reconstructions[mapping.text_d]
        )
        judge_prompt = prompt_maker.render(judge_config)
        
        # Judge the SAME reconstructions with the SAME prompt multiple times
        for judge_run in range(n_judge_runs):
            if store.has_judgment(sample_id, reconstruction_run, judge_run):
                print(f"    Judge run {judge_run}: ✓ Already judged (skipping)")
                continue
            
            print(f"    Judge run {judge_run}: Judging...", end=" ")
            
            # Get structured JSON judgment with schema enforcement
            try:
                response = judge_llm.complete_with_schema(judge_prompt, StyleJudgmentComparative)
                judgment = response.content  # Already validated Pydantic instance
                
                # Save judgment with both reconstruction_run and judge_run
                store.save_judgment(
                    sample_id=sample_id,
                    reconstruction_run=reconstruction_run,
                    judgment=judgment,
                    mapping=mapping,
                    judge_model=response.model,
                    judge_run=judge_run
                )
                print(f"✓ (confidence: {judgment.confidence})")
                
            except Exception as e:
                print(f"✗ Error: {e}")

stats = store.get_stats()
print(f"\n✓ Completed {stats['n_judgments']} judgments")

## Results Analysis

In [ ]:
# Export results to DataFrame
print("=== Exporting Results ===\n")

# Export from store (resolves anonymous rankings to methods)
df = store.to_dataframe()

print(f"Total judgments: {len(df)}")
print(f"Samples: {df['sample_id'].nunique()}")
print(f"Reconstruction runs per sample: {df.groupby('sample_id')['reconstruction_run'].nunique().mean():.1f}")
print(f"Judge runs per reconstruction: {df.groupby(['sample_id', 'reconstruction_run'])['judge_run'].nunique().mean():.1f}")

# Show first few rows (with dynamic method columns)
print(f"\n=== Sample Results ===\n")
display_cols = ['sample_id', 'reconstruction_run', 'judge_run'] + [f'ranking_{m}' for m in METHODS] + ['confidence']
print(df[display_cols].head(25))

# Export to CSV
output_file = f"style_eval_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
df.to_csv(output_file, index=False)
print(f"\n✓ Results saved to {output_file}")

In [103]:
# Analyze judge consistency across multiple judgments of same reconstructions
print("=== Judge Consistency Analysis ===\n")

if 'judge_run' in df.columns and df['judge_run'].nunique() > 1:
    # For each (sample_id, reconstruction_run), check ranking variance across judge_runs
    consistency_results = []
    
    for (sample_id, recon_run), group in df.groupby(['sample_id', 'reconstruction_run']):
        if len(group) > 1:  # Only if multiple judge runs exist
            for method in ['generic', 'fewshot', 'author', 'instructions']:
                col = f'ranking_{method}'
                ranks = group[col].values
                variance = ranks.std()
                mean_rank = ranks.mean()
                
                consistency_results.append({
                    'sample_id': sample_id,
                    'reconstruction_run': recon_run,
                    'method': method,
                    'mean_rank': mean_rank,
                    'std_dev': variance,
                    'min_rank': ranks.min(),
                    'max_rank': ranks.max(),
                    'rank_range': ranks.max() - ranks.min(),
                    'n_judgments': len(ranks)
                })
    
    if consistency_results:
        consistency_df = pd.DataFrame(consistency_results)
        
        print("Method-level consistency (across all samples):")
        method_consistency = consistency_df.groupby('method').agg({
            'std_dev': 'mean',
            'rank_range': 'mean'
        }).round(2)
        method_consistency.columns = ['Avg Std Dev', 'Avg Rank Range']
        print(method_consistency)
        print("\n(Lower values = more consistent)")
        
        print("\n\nMost inconsistent cases (judge disagreed most):")
        inconsistent = consistency_df.nlargest(10, 'rank_range')[
            ['sample_id', 'reconstruction_run', 'method', 'min_rank', 'max_rank', 'rank_range']
        ]
        print(inconsistent.to_string(index=False))
        
        print("\n\nMost consistent cases (judge agreed most):")
        consistent = consistency_df.nsmallest(10, 'std_dev')[
            ['sample_id', 'reconstruction_run', 'method', 'mean_rank', 'std_dev']
        ]
        print(consistent.to_string(index=False))
        
else:
    print("⚠ Only one judge_run per reconstruction. Set n_judge_runs > 1 to test consistency.")

=== Judge Consistency Analysis ===

⚠ Only one judge_run per reconstruction. Set n_judge_runs > 1 to test consistency.


In [105]:
print(df.loc[0,'reasoning'])

To rank these texts, I focused on how closely each reconstruction captures the original author’s distinctive voice—its rhythm, cadence, word choice, and the relationship it establishes with the reader. The original text is characterized by a measured, philosophical tone that balances intellectual rigor with a conversational accessibility. It often employs long, flowing sentences that build complex arguments, yet it avoids pretension through direct address (e.g., "I do not mean that...") and a subtle dry wit. The author’s voice is authoritative but not dogmatic, inviting the reader into a shared exploration of ideas rather than lecturing. There’s also a rhythmic quality to the prose, with clauses that accumulate evidence or counterarguments in a deliberate, almost musical progression. The original also exhibits a particular kind of moral urgency, especially around democratic ideals, without slipping into polemic. It’s this combination of intellectual depth, stylistic elegance, and democ

In [ ]:
# Analyze ranking distributions (dynamic for configured methods)
print("=== Ranking Distribution by Method ===\n")

for method in METHODS:
    col = f'ranking_{method}'
    print(f"\n{method.upper()}:")
    ranking_counts = df[col].value_counts().sort_index()
    for rank in [1, 2, 3, 4]:
        count = ranking_counts.get(rank, 0)
        pct = (count / len(df) * 100) if len(df) > 0 else 0
        print(f"  Rank {rank}: {count:3d} ({pct:5.1f}%)")

print("\n=== Confidence Distribution ===\n")
print(df['confidence'].value_counts())

In [ ]:
# Calculate method performance metrics (dynamic for configured methods)
print("=== Method Performance Metrics ===\n")

# Calculate mean ranking for each method (lower is better: 1 = best, 4 = worst)
mean_rankings = {}
for method in METHODS:
    col = f'ranking_{method}'
    mean_rankings[method] = df[col].mean()

# Sort by mean ranking (best first)
sorted_methods = sorted(mean_rankings.items(), key=lambda x: x[1])

print("Average Ranking (lower is better):")
for i, (method, mean_rank) in enumerate(sorted_methods, 1):
    # Count how often this method ranked 1st
    first_place = (df[f'ranking_{method}'] == 1).sum()
    first_place_pct = (first_place / len(df) * 100) if len(df) > 0 else 0
    
    print(f"{i}. {method:15s}: {mean_rank:.2f} (1st place: {first_place}/{len(df)} = {first_place_pct:.1f}%)")

# Win rate (percentage of times ranked 1st or 2nd)
print("\nTop-2 Rate (ranked 1st or 2nd):")
for method in METHODS:
    col = f'ranking_{method}'
    top2 = ((df[col] == 1) | (df[col] == 2)).sum()
    top2_pct = (top2 / len(df) * 100) if len(df) > 0 else 0
    print(f"  {method:15s}: {top2}/{len(df)} = {top2_pct:.1f}%")

In [25]:
# Export results
output_file = f"style_evaluation_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
df.to_csv(output_file, index=False)
print(f"\n✓ Results saved to {output_file}")


✓ Results saved to style_evaluation_results_20251202_164845.csv


## Next Steps

TODO: Add analysis cells for:
- Statistical significance testing
- Visualization of results
- Sample-by-sample breakdown
- Confidence level analysis
- Qualitative review of reasoning